#### TOBIG'S 14기 정규세션 4주차 SVM 
### ASSIGNMENT1. Multiclass SVM 구현

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [40]:
scal = StandardScaler() #scaling
X = scal.fit_transform(X)

In [41]:
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [42]:
# One VS Rest 구현을 위해 1) class가 0 or not 2)class가 1 or not을 구분하기 위한 머신 등을 미리 만들어주자
svm_1 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_2 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_3 = SVC(kernel ='rbf', C = 5, gamma = 5)

In [43]:
from sklearn.model_selection import train_test_split #test/train 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [44]:
y_train = pd.get_dummies(y_train) #one hot encoding

In [46]:
svm_1.fit(X_train,y_train.iloc[:,0]) # 데이터 클레스가 0 or not 구분해주는 머신
svm_2.fit(X_train,y_train.iloc[:,1]) # 데이터 클레스가 1 or not 구분해주는 머신
svm_3.fit(X_train,y_train.iloc[:,2]) # 데이터 클레스가 2 or not 구분해주는 머신
print(svm_1.predict(X_test)) #데이터 클래스가 0 or not을 구분해주는 애를 통해서 테스트 데이터의 클래스가 0인지, 0이 아닌인지 예측해보자

print(svm_1.decision_function(X_test)) #decision_function hyperplane과의 거리를 구하는 방법(필요하다면 활용해주세요!)

[0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[-1.12470845 -0.86326953 -0.65281099 -0.50248821 -0.76284323 -0.87465573
  1.07709345 -0.99281647  0.47441336 -0.99842743 -0.83989348  0.15633457
  0.32871788 -0.97965464 -0.72385083 -0.92638376  1.28322481 -0.56240455
 -0.72719892 -0.99509775  0.43166724 -0.96451557 -0.82991366 -1.03020581
 -0.75166835  1.13461335  0.39943974 -1.04194106 -0.93376548 -1.06133798]


In [57]:
svm_1.decision_function(X_test)[3], svm_2.decision_function(X_test)[3], svm_3.decision_function(X_test)[3]

(-0.5024882107322184, -0.36987092513822756, -0.13070947139128608)

In [47]:
# 부호가 모든 같은 경우가 있는가? > 모두 동점인 경우가 생길 것
for i in range(len(X_test)):
    # ~. decision_function을 이용하면 해당 데이터가 하이퍼플레인으로부터 얼마나 떨어져있는지 '거리'가 나온다!
    # 다음은 그 값의 부호를 이용해 모두가 동점인 경우가 있는지 출력하는 함수 
    if (np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i])) and (np.sign(svm_2.decision_function(X_test)[i]) == np.sign(svm_3.decision_function(X_test)[i])):
        print(i)

3
17
18


In [70]:
## Case 1 : One vs Rest SVM을 이부분에 구현해주세요 위 결과들을 이용해서 multi class SVM을 직접 구현해주세요! 하드코딩이 하시기 편할겁니다.

def one_vs_rest_svm(X,y) :
    pred_classes = np.zeros((len(X),3))
    length = len(X)
    y_label = {0 : 'setosa', 1 : 'versicolor', 2 : 'virginica'}
    results = []
    
    # 3개의 분류기 중 결과값이 1인 분류기에 +1을 하여 진행 
    for i in range(length)  :
        if svm_1.predict(X)[i] == 1 :
            pred_classes[i][0] += 1
        elif svm_2.predict(X)[i] == 1 :
            pred_classes[i][1] += 1
        elif svm_3.predict(X)[i] == 1 :
            pred_classes[i][2] += 1
        
        # 0 : 0 : 0 인 경우 decision_function을 이용하여 가까운 거리의 경우 그 분류 값에 해당하게 설정을 한다. 
        if pred_classes[i][0] == pred_classes[i][1] and pred_classes[i][1] == pred_classes[i][2] :
            results.append(np.argmax([svm_1.decision_function(X)[i], svm_2.decision_function(X)[i], svm_3.decision_function(X)[i]]))
        else :
            results.append(np.argmax(pred_classes[i]))
        
    
    results = [y_label.get(i) for i in results]    
    
    #print(pred_classes)
    print(results)
    
    
    score = accuracy_score(y, results)
    
    return score
    

- 이 one vs rest svm의 경우 분류기 하나씩 투표를 하여 가장 많이 나오는 경우 해당 class를 부여하는 방식으로 진행하였다.
- 예외의 경우 0 : 0 : 0 일 때는 argmax한 결과 0번째 index가 나와 올바르게 분류하기 힘듦으로 decision_function을 이용하여 거리를 비교하여 argmax를 진행하는데
- 이러한 방식을 이용하여 밑의 코드와 같이 간단하게 정리할 수  있다.

In [71]:
def one_vs_rest_svm_1(X,y) :
    length = len(X)
    y_label = {0 : 'setosa', 1 : 'versicolor', 2 : 'virginica'}
    results = []
    
    
    for i in range(length)  :
            results.append(np.argmax([svm_1.decision_function(X)[i], svm_2.decision_function(X)[i], svm_3.decision_function(X)[i]]))
        
    
    results = [y_label.get(i) for i in results]    
    
    
    print(results)
    
    
    score = accuracy_score(y, results)
    
    return score
    

In [72]:
one_vs_rest_svm(X_test,y_test)

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


0.8666666666666667

In [73]:
one_vs_rest_svm_1(X_test,y_test)

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


0.8666666666666667

- 위와 아래의 결과가 같음을 확인할 수 있다.

In [260]:
# Case 2 : One vs One SVM을 이 부분에 구현해주세요. (선택사항)

# 'setosa', 'virginica', 'versicolor' 세 class에 대하여 분류를 해야 하기 때문에 
# 데이터 셋을 setosa vs virginca / setosa vs vesicolor / virginica vs versicolor로 나누어야 한다.
# 그래서 다음과 같이 함수를 지정하였다.
def get_new_data(X,y,class_1, class_2) :
    idx_1 = y_train[class_1] == 1
    idx_2 = y_train[class_2] == 1
    
    new_y = np.concatenate((y[idx_1][class_1],y[idx_2][class_1]))
    
    new_X = np.vstack((X[idx_1],X[idx_2]))
    
    return new_X, new_y
    
    

def one_vs_one_svm(X_train, y_train, X_test, y_test) :
    columns = list(y_train.columns)
    svm_clfs = []
    for i in range(len(columns)) :
        for j in range(i+1,len(columns)) :
            # 새로운 데이터셋에 대하여 각각 svm을 돌려 class를 예측한다.
            # 이경우 3개의 분류값이 존재하므로 총 3C2 개수의 분류기를 생성하여 저장한다.
            new_X, new_y = get_new_data(X_train, y_train, columns[i], columns[j])

            svm = SVC(kernel ='rbf', C = 5, gamma = 5)

            svm.fit(new_X, new_y)
            svm_clfs.append([svm, columns[i], columns[j]])

    pred_classes = np.zeros((len(X_test),3))
    length = len(X_test)

    y_label = {0 : 'setosa', 1 : 'versicolor', 2 : 'virginica'}
    r_y_label = {'setosa' : 0, 'versicolor' : 1, 'virginica' : 2}

    results = []

    # test 데이터를 각각 분류기에 적용시켜 해당하는 class에 +1을 지정해준다. 
    # 편의상 columns 순성에 맞게 0 : setosa, 1 : versicolor, 2 : virginica를 해주어 가장 큰 값을 지니면 해당 index의 label를 return 해준다.
    for k in range(length) :
        for l in range(len(svm_clfs)) :
            svm, c1, c2 = svm_clfs[l]
            pred = svm.predict([X_test[k]])

            if pred == 1 :
                pred_classes[k][r_y_label.get(c1)] += 1
            else :
                pred_classes[k][r_y_label.get(c2)] += 1
        
        # 모두 1 : 1 : 1 로 같은 경우를 고려하여  desicion_function을 적용하여 return해준다.
        if pred_classes[k][0] == pred_classes[k][1] and pred_classes[k][1] == pred_classes[k][2] :
            results.append(np.argmax([svm_clfs[0][0].decision_function([X_test[k]]), svm_clfs[1][0].decision_function([X_test[k]]), svm_clfs[2][0].decision_function([X_test[k]])]))
        else :
            results.append(np.argmax(pred_classes[k]))


    results = [y_label.get(i) for i in results]  
    
    print(results)

    score = accuracy_score(y_test, results)
    
    return score


In [261]:
one_vs_one_svm(X_train,y_train, X_test, y_test)

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


0.8333333333333334

In [269]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요
from sklearn.model_selection import train_test_split #데이터셋 분리
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

print(y_pred)
accuracy_score(y_test_2,y_pred)

['versicolor' 'versicolor' 'versicolor' 'virginica' 'virginica'
 'virginica' 'setosa' 'virginica' 'setosa' 'versicolor' 'virginica'
 'setosa' 'setosa' 'virginica' 'versicolor' 'versicolor' 'setosa'
 'virginica' 'virginica' 'virginica' 'setosa' 'virginica' 'versicolor'
 'versicolor' 'virginica' 'setosa' 'setosa' 'virginica' 'virginica'
 'versicolor']


0.8333333333333334

In [270]:
one_vs_rest_svm_1(X_test_2, y_test_2)

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


0.8666666666666667

In [271]:
y_train_2 = pd.get_dummies(y_train_2)

one_vs_one_svm(X_train_2, y_train_2, X_test_2, y_test_2)

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


0.8333333333333334

- 결과를 비교해보면 one vs one svm은 동일한 결과가 나오는 반면에 one vs rest svm은 보다 좋은 성능이 나오는 것을 확인할 수 있다.